In [3]:
import ray
from ray import train, tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
ray.init(ignore_reinit_error=True)

config = PPOConfig().training(lr=tune.grid_search([0.01, 0.001, 0.0001]))

tuner = tune.Tuner(
    "PPO",
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 150},
    ),
    param_space=config,
)

tuner.fit()

2024-05-13 23:28:06,658	ERROR tune_controller.py:1331 -- Trial task failed for trial PPO_None_5cb3c_00000
Traceback (most recent call last):
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
             ^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/_private/worker.py", line 2623, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeou

ResultGrid<[
  Result(
    error='ActorDiedError',
    metrics={},
    path='/home/jianghaoning/ray_results/PPO_2024-05-13_23-27-51/PPO_None_5cb3c_00000_0_lr=0.0100_2024-05-13_23-27-51',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='ActorDiedError',
    metrics={},
    path='/home/jianghaoning/ray_results/PPO_2024-05-13_23-27-51/PPO_None_5cb3c_00001_1_lr=0.0010_2024-05-13_23-27-51',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    error='ActorDiedError',
    metrics={},
    path='/home/jianghaoning/ray_results/PPO_2024-05-13_23-27-51/PPO_None_5cb3c_00002_2_lr=0.0001_2024-05-13_23-27-51',
    filesystem='local',
    checkpoint=None
  )
]>

In [10]:
# Note: `gymnasium` (not `gym`) will be **the** API supported by RLlib from Ray 2.3 on.
try:
    import gymnasium as gym

    gymnasium = True
except Exception:
    import gym

    gymnasium = False

from ray.rllib.algorithms.ppo import PPOConfig

env_name = "CartPole-v1"
env = gym.make(env_name)
algo = PPOConfig().environment(env_name).build()

episode_reward = 0
terminated = truncated = False

if gymnasium:
    obs, info = env.reset()
else:
    obs = env.reset()

while not terminated and not truncated:
    action = algo.compute_single_action(obs)
    if gymnasium:
        obs, reward, terminated, truncated, info = env.step(action)
    else:
        obs, reward, terminated, info = env.step(action)
    episode_reward += reward

/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/tune/logg

/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/models/catalog.py:895: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prep = cls(observation_space, options)
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/connectors/agent/obs_preproc.py:37: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  self._preprocessor = get_preprocessor(obs_space)(
2024-05-13 23:42:25,194	WARNING util.py:61 -- Install gputil for GPU system monitoring.


如何在RLlib中访问策略状态（policy state），即算法内部的状态，例如设置或获取模型权重等操作。

首先，它指出RLlib算法的状态是在集群中的多个回滚工作者（Rollout Workers）之间复制的。回滚工作者是Ray的actor，负责执行环境交互和生成训练数据。然后，它介绍了两种方法来在调用train()之间轻松获取和更新此状态：

使用Algorithm.workers.foreach_worker()或Algorithm.workers.foreach_worker_with_index()函数。这些函数接受一个lambda函数作为参数，并将其应用于每个回滚工作者。它们返回每个工作者的值列表，允许您在训练周期之间进行状态的传递和更新。

通过Algorithm.get_policy()或Algorithm.workers.local_worker()访问算法状态的“主”副本。这允许您直接访问主副本的状态，但需要注意，对主副本的更新可能不会立即反映在回滚工作者上（如果您配置了num_env_runners > 0）。这意味着如果您在num_env_runners > 0的情况下进行了更新，可能需要一些时间才能在回滚工作者中看到更新。

In [11]:
from ray.rllib.algorithms.dqn import DQNConfig

algo = DQNConfig().environment(env="CartPole-v1").build()

# Get weights of the default local policy
algo.get_policy().get_weights()

# Same as above
algo.workers.local_worker().policy_map["default_policy"].get_weights()

# Get list of weights of each worker, including remote replicas
algo.workers.foreach_worker(lambda worker: worker.get_policy().get_weights())

# Same as above, but with index.
algo.workers.foreach_worker_with_id(
    lambda _id, worker: worker.get_policy().get_weights()
)

2024-05-13 23:43:03,478	WARNING deprecation.py:50 -- DeprecationWarning: `WorkerSet(num_workers=... OR local_worker=...)` has been deprecated. Use `EnvRunnerGroup(num_env_runners=... AND local_env_runner=...)` instead. This will raise an error in the future!
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/algorithms/dqn/dqn_torch_model.py:61: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  super(DQNTorchModel, self).__init__(
2024-05-13 23:43:03,768	WARNING util.py:61 -- Install gputil for GPU system monitoring.


[{'_hidden_layers.0._model.0.weight': array([[-0.23787265, -0.32587072,  0.10094426, -0.9094147 ],
         [ 0.46850386,  0.12140537,  0.38471243,  0.78597784],
         [ 0.29163662, -0.21202134,  0.58655345,  0.7252241 ],
         ...,
         [-0.6381535 , -0.25386614, -0.36334828,  0.62951577],
         [ 0.07889429,  0.9354121 ,  0.22815892,  0.25830847],
         [ 0.60462385,  0.0169113 , -0.58495396, -0.54034513]],
        dtype=float32),
  '_hidden_layers.0._model.0.bias': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

如何访问模型状态，类似于访问策略状态，你可能想要获取正在训练的底层神经网络模型的引用。例如，你可能希望单独对其进行预训练，或以其他方式在RLlib之外更新其权重。这可以通过访问策略的模型来实现。

简而言之，当你想要对正在训练的神经网络模型进行额外操作时，比如独立地进行预训练或者在RLlib之外更新权重时，你可以通过访问策略（policy）的模型来实现这一目的。这允许你直接操作模型，而无需依赖RLlib的训练循环。

In [14]:
try:
    import gymnasium as gym

    env = gym.make("CartPole-v1")
    obs, infos = env.reset()
except Exception:
    import gym

    env = gym.make("PongNoFrameskip-v4")
    obs = env.reset()

# RLlib uses preprocessors to implement transforms such as one-hot encoding
# and flattening of tuple and dict observations.
from ray.rllib.models.preprocessors import get_preprocessor

prep = get_preprocessor(env.observation_space)(env.observation_space)
# <ray.rllib.models.preprocessors.GenericPixelPreprocessor object at 0x7fc4d049de80>

# Observations should be preprocessed prior to feeding into a model
obs.shape
# (210, 160, 3)
prep.transform(obs).shape
# (84, 84, 3)

/tmp/ipykernel_40706/627519884.py:16: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prep = get_preprocessor(env.observation_space)(env.observation_space)


(4,)

In [1]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.callbacks import MemoryTrackingCallbacks
# Construct a generic config object, specifying values within different
# sub-categories, e.g. "training".
config = (PPOConfig().training(gamma=0.9, lr=0.01)
        .environment(env="CartPole-v1")
        .resources(num_gpus=0)
        .env_runners(num_env_runners=0)
        .callbacks(MemoryTrackingCallbacks)
    )
# A config object can be used to construct the respective Algorithm.
rllib_algo = config.build()

/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/simulation/jianghaoning/anaconda3/envs/multi_agent/lib/python3.11/site-packages/ray/tune/logg

In [2]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray import tune
# In combination with a tune.grid_search:
config = PPOConfig()
config.training(lr=tune.grid_search([0.01, 0.001]))
# Use `to_dict()` method to get the legacy plain python config dict
# for usage with `tune.Tuner().fit()`.
tune.Tuner("PPO", param_space=config.to_dict())

Training options

In [ ]:
""" 
Parameters:
gamma (float) : The discount factor of the MDP.
lr (float) : The learning rate of the policy.
grad_clip (float) : The gradient clipping value.
grad_clip_by(str) : The gradient clipping method.
train_batch_size_per_learner (int) : The batch size for training.
train_batch_size (int) : The total batch size for training.
model(dict) : The model configuration.
optimizer(dict) : The optimizer configuration.
max_request_in_flight_per_sampler_worker (int) : The maximum number of requests in flight per sampler worker.
learner_class
learner_connector
add_default_connectors_to_learner_pipeline
"""

Specifying environments

In [ ]:
""" 
Parameters:
env(str) : The environment name.
env_config
observation_space
action_space
env_task_fn
render_env
clip_rewards 
normalize_actions
clip_actions
is_atari 
action_mask_key 

"""